# Vector data
This lesson will cover Vector data, giving some theoretical background, introduce
Vector storage formats and finally explore Python libraries for Vector data access and manipulation.

## Background reading

* https://docs.qgis.org/3.16/en/docs/gentle_gis_introduction/vector_data.html

## What is vector data?
Vector data is spatial data, generally consisting of two parts: 

* Geometry
* Attributes

**Geometries** are the *Points, Lines and Polygons* as introduced in the [Geometries Lesson](02-geometry.ipynb).
They represent the "shape" of the real-world phenomenon. 
**Attribute** data is information appended to the Geometry (or the other way around) 
usually in tabular format ("records"). Together, this combination Geometry+Attributes 
is often called a (Spatial) **Feature**.

![Vector Data in QGIS](images/qgis-attr-table.png)

A [Triangulated Irregular network (TIN)](https://en.wikipedia.org/wiki/Triangulated_irregular_network) 
is also an example of Vector data.

## Vector data formats
There are currently [over 100 vector data formats](https://gdal.org/drivers/vector/index.html) used for storage, e.g. files, and for data transfer.
The most common formats are presented below. 

> Tip: [ogr2ogr](https://gdal.org/programs/ogr2ogr.html) is a GDAL/OGR commandline utility
> that allows you to convert between most vector formats.  

### ESRI Shapefile

[ESRI Shapefile](https://en.wikipedia.org/wiki/Shapefile) is a file-based format. It consists of at least 3 files:

* .shp containing geometry
* .shx containing index
* .dbf attribute table

The ESRI Shapefile is one of the oldest formats, some even call it a [Curse in Geoinformatics](https://www.slideshare.net/jachym/switch-from-shapefile), and is more and more replaced by GeoPackage.

### GeoPackage

[GeoPackage](https://www.geopackage.org/) is a relatively new but promising spatial data format based on [SQLite](https://www.sqlite.org).

The [OGC GeoPackage Encoding Standard](https://www.opengeospatial.org/standards/geopackage) describes a set of conventions for storing the following 
within an SQLite database:
  
* vector features
* tile matrix sets of imagery and raster maps at various scales
* attributes (non-spatial data)
* extensions
  
Thus GeoPackage can store vector as well as raster data. GeoPackage is by some called "The Shapefile Killer".
We recommend using GeoPackage over ESRI Shapefile.

### GeoJSON

[GeoJSON](https://geojson.org) is a simple JSON-based format to encode vector Features. 
It is increasingly popular, especially among web developers. 

Example:

```
{
  "type": "Feature",
  "geometry": {
    "type": "Point",
    "coordinates": [125.6, 10.1]
  },
  "properties": {
    "name": "Dinagat Islands"
    "population": 4785
  }
}
```

GitHub for example is able to display [GeoJSON-encoded data on-the-fly](https://github.com/jachym/jrdata/blob/master/jsons/stops.geojson).

### Geography Markup Language (GML)

> The Geography Markup Language (GML) is the XML grammar defined by the Open Geospatial Consortium (OGC) 
> to express geographical features. GML serves as a modeling language for geographic 
> systems as well as an open interchange format for geographic transactions on the Internet. Source: [Wikipedia](https://en.wikipedia.org/wiki/Geography_Markup_Language).

Below an example of the same feature we saw earlier as GeoJSON, now in GML:

```
<gml:featureMember>
  <feature fid="12">
	<id>23</id>
	<name>Dinagat Islands</name>
	<population>4785</population>
	<ogr:geometry>
	  <gml:Point gml:id="p21" srsName="http://www.opengis.net/def/crs/EPSG/0/4326">
        <gml:pos srsDimension="2">125.6, 10.1</gml:pos>
      </gml:Point>
	</ogr:geometry>
  </feature>
</gml:featureMember>
```

GML is defined as a joint ISO-OGC Standard:

> ISO 19136 Geographic information – Geography Markup Language, is a standard from the family 
> ISO - of the standards for geographic information (ISO 191xx). It resulted from unification 
> of the Open Geospatial Consortium definitions and Geography Markup Language (GML) with 
> the ISO-191xx standards. Source: [Wikipedia](https://en.wikipedia.org/wiki/Geography_Markup_Language)

*GML Application Schemas* adds a convention to the GML standard to define domain- or community- specific application
models. Examples are [CityGML](https://en.wikipedia.org/wiki/CityGML) and schemas developed within [INSPIRE](https://inspire.ec.europa.eu/applicationschema).

GML sees quite widespread use, but due to its complexity (e.g. multiple encodings for coordinates and projections) and verbosity, is more and more
replaced by GeoJSON.

### CSV

Of course, you  can save your data in a comma separated values text file.

### PostgreSQL/PostGIS database

[PostGIS](https://postgis.net) adds support for geographic objects to the PostgreSQL object-relational database. 
In effect, PostGIS "spatially enables" the PostgreSQL server, allowing it to be 
used as a backend spatial database for geographic information systems (GIS), 
much like ESRI's SDE or Oracle's Spatial extension. 
PostGIS follows the OGC [Simple Features Specification for SQL](https://www.opengeospatial.org/standards/sfs) 
and has been certified as compliant with the "Types and Functions" profile. 

Like said, there are [many more vector formats](https://gdal.org/drivers/vector/index.html).

## Vector libraries
Within Python there is an ample choice of libraries to interact with vector data. The
most popular are:

* [Python bindings](https://gdal.org/python/) for [GDAL OGR](https://gdal.org/), a.k.a. "OGR"
* [Fiona](https://toblerity.org/fiona/manual.html) 
* [GeoPandas](https://geopandas.org/) 
 
This chapter will first focus on Fiona and OGR, ending with GeoPandas.
[Fiona](https://toblerity.org/fiona/) is maintained by [Sean Gillies](https://github.com/sgillies) and adds a utility/wrapper layer on top of OGR in a Pythonic fashion.
Compared to Fiona, OGR (Python bindings) provides more finegrained control over data, for example reprojections,
and supports all GDAL/OGR vector formats.

## Manipulating features with Fiona and Shapely
Fiona and Shapely are often used together.
Here we use Fiona 
to read Vector data (Features) into memory for subsequent manipulation with Shapely.

Feature geometry can be accessed using the `geometry` property of each feature. For example
we can open the dataset that contains a (Multi)Polygon for each country and print
out the geometry of a random Feature (country):

First we import `Shapely` and its functions and then convert the JSON-encoded geometries to Geometry objects
using the `shape` function.

In [ ]:
import fiona
from shapely.geometry import shape


Next we open a GeoPackage `countries` file and loop through the Features.
You may observe the Pythonism that Fiona supplies (using `with` and `as`) to
open and loop through Features in a single step.

  > NB the countries-file has its geometries in SRS/CRS EPSG:3857, also known as the
  > [Web Mercator Projection](https://en.wikipedia.org/wiki/Web_Mercator_projection). That projection is
  > in meters, hence values like `area` are in meters as well.

In [ ]:
with fiona.open("../data/countries.3857.gpkg") as countries:
	country = countries[4]
	print("This is %s" % country["properties"]["NAME"])
	geom = shape(country["geometry"])
  
geom # Jupyter can display geometry data directly

In [ ]:
print(geom.type)

In [ ]:
print(geom.area)

In [ ]:
# In km
print(geom.length/1000)

Let's have a look at some geometry methods. 
Tip: Shapely code is well-documented, you can always use the Python built-in `help()` function.

In [ ]:
help(geom)

For example we can make a buffer of 500 meter around our polygon (making Canada somewhat bigger):

In [ ]:
buffered_geom = geom.buffer(500)
buffered_geom

In [ ]:
# In km
buffered_geom.length/1000

### Converting the geometry back to JSON format
Once we are finished, we can convert the geometry back to JSON format using `shapely.geometry.mapping` function


In [ ]:
from shapely.geometry import mapping

In [ ]:
# let's create new GeoJSON-encoded vector feature

new_feature = {
	"type": "Feature",
	"properties": {"name": "My buffered feature"},
	"geometry": mapping(buffered_geom)
}
new_feature

# Now we could e.g. write the Feature back to file


## GDAL/OGR Python Bindings


[OGR](https://gdal.org/faq.html#what-is-this-ogr-stuff) is part of the [GDAL](https://gdal.org/) library for the support of Vector data. 
OGR supports about [100+ vector formats](https://gdal.org/drivers/vector/index.html) and
has more/other functionalities (than Fiona) like reprojection.

The OGR API wraps differences between various vector formats, web-services, database etc..
The following terminology applies to OGR:

* **Driver** - driver for reading and writing for a specified format
* **Data Source** - the named data source (file, database, web-service, ...)
* **Layer** - data layer within the Data Source (file content, database table, ...)
* **Feature** - vector feature
* **Field, Geometry** - attributes and geometry

The OGR-Python interface is an abstract API on top of the 
original classes and methods of the original C++ code. 
Because of this, some approaches may seem complicated, 
compared to native Python code, like e.g. Fiona.

### Links

* GDAL OGR Vector API tutorial: https://gdal.org/tutorials/vector_api_tut.html
* Python API: https://gdal.org/python/
* GDAL/OGR Python Cookbook https://pcjericks.github.io/py-gdalogr-cookbook/ - Recommended!

### Buffer
First we need to open the *Data Source*, printing the number of Layers.

In [ ]:
from osgeo import ogr
ds = ogr.Open("../data/countries.gpkg")
print(ds)
print(ds.GetLayerCount())

Next we have to fetch and open the *Layer*. NB for files, there is usually just one layer, index `0`, 
within the Data Source (DS), but for example for a database DS, a Layer is refers to a concrete table).

In [ ]:
l = ds.GetLayer(0)
print(l)
print(l.GetFeatureCount())

Show the schema of the layer and the definition of its geometry type:

In [ ]:
l.GetGeomType()

In [ ]:
l.GetGeomType() == ogr.wkbMultiPolygon

In [ ]:
for s in l.schema:
    print(s.GetName())

In [ ]:
l.schema[4].name

Print name attribute of all features

In [ ]:
features_nr = l.GetFeatureCount()
for i in range(features_nr):
    f = l.GetNextFeature()
    print(f.GetField('NAME'))

Get vector feature bounding box (envelope):

In [ ]:
f = l.GetFeature(4)
geom = f.GetGeometryRef()
geom.GetEnvelope()

Get geometry centroid

In [ ]:
c = geom.Centroid()
c.GetPoint()

Get geometry buffer

In [ ]:
buff = c.Buffer(100)
geom.Intersects(buff)

### Complete example

In this example we will demonstrate working with vector data from begin to
end: open a data set, metadata, attribute change, saving of new attribute 
back to the file. 

In [ ]:
from osgeo import osr

# Creating new file with new driver
drv = ogr.GetDriverByName('GML')
ds = drv.CreateDataSource('../data/04-ogr-out.gml')
srs = osr.SpatialReference()
srs.ImportFromEPSG(3857)
print(srs.ExportToProj4())
layer = ds.CreateLayer('outgml', srs, ogr.wkbLineString)

# create new attributes named and code
field_name = ogr.FieldDefn('name', ogr.OFTString)
field_name.SetWidth(24)
field_number = ogr.FieldDefn('code', ogr.OFTInteger)
layer.CreateField(field_name)
layer.CreateField(field_number)

# create new line geometry and read from WKT
line = ogr.CreateGeometryFromWkt('LINESTRING(%f %f, %f %f)' % (0, 0, 1, 1))

# create new feature, set attributes and geometry
feature = ogr.Feature(layer.GetLayerDefn())
feature.SetGeometry(line)
feature.SetField("name", 'the line')
feature.SetField("code", 42)

layer.CreateFeature(feature)

# final cleaning
feature.Destroy()
ds.Destroy()

now we can check the result

In [ ]:
ds = ogr.Open('../data/04-ogr-out.gml')
layer = ds.GetLayer(0)
print(layer.GetFeatureCount())
print(layer.GetFeature(0).GetField('name'))
print(layer.GetFeature(0).GetField('code'))
ds.Destroy()

## Fiona or GDAL/OGR?
With Fiona, the above example would be much simpler and Pythonic. 
However, OGR accesses the data on a much lower/efficient level compared to Fiona, 
therefore bigger datasets can be more easily handled. Also OGR supports more data formats and
functionality like reprojection.

We recommend to have both Fiona (plus Shapely) and OGR in your toolbox
and assess at project-time which to apply.

## Introduction to GeoPandas

Geospatial data is often available from specific GIS file formats or data stores, like ESRI shapefiles, GeoJSON files, geopackage files, PostGIS (PostgreSQL) database, ...

We can use the [GeoPandas](https://geopandas.org/) library to read many of those GIS file formats (relying on the `Fiona` library under the hood, which is an interface to GDAL/OGR), using the `geopandas.read_file` function.

For example, let's start by reading a GeoPackage with all the countries of the world, and inspect the data:

In [ ]:
%matplotlib inline

import pandas as pd
import geopandas

pd.options.display.max_rows = 10

In [ ]:
# Ignore the warning: https://stackoverflow.com/questions/64995369/geopandas-warning-on-read-file
countries = geopandas.read_file("../data/countries.gpkg")

In [ ]:
countries.head()

In [ ]:
countries.plot()

### What's a GeoDataFrame?

We used the GeoPandas library to read in the geospatial data, and this returned us a `GeoDataFrame`:

In [ ]:
type(countries)

A GeoDataFrame contains a tabular, geospatial dataset:

* It has a **'geometry' column** that holds the geometry information (or features in GeoJSON).
* The other columns are the **attributes** (or properties in GeoJSON) that describe each of the geometries

Such a `GeoDataFrame` is just like a pandas `DataFrame`, but with some additional functionality for working with geospatial data:

* A `.geometry` attribute that always returns the column with the geometry information (returning a GeoSeries). The column name itself does not necessarily need to be 'geometry', but it will always be accessible as the `.geometry` attribute.
* It has some extra methods for working with spatial data (area, distance, buffer, intersection, ...), which we will see in later notebooks

In [ ]:
countries.geometry

In [ ]:
type(countries.geometry)

In [ ]:
countries.geometry.area

*Question: are the above areas correct? Why (not)?*

**It's still a DataFrame**, so we have all the Pandas functionality available to use on the geospatial dataset, and to do data manipulations with the attributes and geometry information together.

For example, we can calculate average population number over all countries (by accessing the 'POP_EST' column, and calling the `mean` method on it):

In [ ]:
countries['POP_EST'].mean()

Or assemble and plot all countries from the African continent.

In [ ]:
africa = countries[countries['CONTINENT'] == 'Africa']

In [ ]:
africa.plot()

### Spatial relationships with GeoDataFrames

We can apply data filtering in GeoPandas based on spatial relationships.

For example, if we call the `contains` method on the world dataset with the `paris` point, it will do this spatial check for each country in the `countries` dataframe:

In [ ]:
cities = geopandas.read_file("../data/populated_places.gpkg")

In [ ]:
paris = cities.loc[cities['NAME'] == 'Paris', 'geometry'].squeeze()

In [ ]:
countries.contains(paris)

Because the above gives us a boolean result, we can use that to filter the dataframe:

In [ ]:
countries[countries.contains(paris)]

---
[<- Spatial Reference Systems](03-spatial-reference-systems.ipynb) | [Raster data ->](05-raster-data.ipynb)
